# One-to-Many and Many-to-Many Joins - Lab

## Introduction

In this lab, you'll practice your knowledge on One-to-Many and Many-to-many relationships!

## Objectives

You will be able to:
- Query data including Many-to-Many relationships
- Write queries that make use of Join Tables

In [1]:
import sqlite3
import pandas as pd

In [2]:
conn = sqlite3.connect('data.sqlite', detect_types=sqlite3.PARSE_COLNAMES)
cur = conn.cursor()

## One-to-Many and Many-to-Many Joins
<img src='Database-Schema.png' width=550>

## Employees and their Office (a One-to-One join)

Return a list of all of the employees with their first name, last name and the city and state of the office that they work out of (if they have one). Include all employees and order them by their first name, then their last name.

In [3]:
cur.execute('''select firstName, lastName, city, state
                from employees
                join 
                offices using(officeCode)
                order by 1, 2;''')
df = pd.DataFrame(cur.fetchall())
df.columns = [i[0] for i in cur.description]
print(len(df))
df.head(10)

23


,firstName,lastName,city,state
0,Andy,Fixter,Sydney,
1,Anthony,Bow,San Francisco,CA
2,Barry,Jones,London,
3,Diane,Murphy,San Francisco,CA
4,Foon Yue,Tseng,NYC,NY
5,George,Vanauf,NYC,NY
6,Gerard,Bondur,Paris,
7,Gerard,Hernandez,Paris,
8,Jeff,Firrelli,San Francisco,CA
9,Julie,Firrelli,Boston,MA


## Customers and their Orders (a One-to-Many join)

Return a list of all of the customers. For each customer return a record for each of their order numbers, order dates and statuses.

In [29]:
cur.execute('''select orderNumber from orders;''')
df = pd.DataFrame(cur.fetchall())
print(len(df))

326


In [5]:
cur.execute('''select customerName from customers;''')
df = pd.DataFrame(cur.fetchall())
print(len(df))
df.head()

122


,0
0,Atelier graphique
1,Signal Gift Stores
2,"Australian Collectors, Co."
3,La Rochelle Gifts
4,Baane Mini Imports


In [38]:
cur.execute('''select customerNumber, count(orderNumber) total_orders
                from orders
                group by 1;''')
df = pd.DataFrame(cur.fetchall())
df.columns = [i[0] for i in cur.description]
print(len(df))
df.head()


98


,customerNumber,total_orders
0,103,3
1,112,3
2,114,5
3,119,4
4,121,4


In [36]:
cur.execute('''select customerName, orderNumber, orderDate, status 
            from orders 
            left join 
            customers using(customerNumber)
            order by 1;''')

df = pd.DataFrame(cur.fetchall())
df.columns = [i[0] for i in cur.description]
print(len(df))
df.head()

326


,customerName,orderNumber,orderDate,status
0,"AV Stores, Co.",10110,2003-03-18,Shipped
1,"AV Stores, Co.",10306,2004-10-14,Shipped
2,"AV Stores, Co.",10332,2004-11-17,Shipped
3,Alpha Cognac,10136,2003-07-04,Shipped
4,Alpha Cognac,10178,2003-11-08,Shipped


## Orders and their Order Details (another One-to-Many join)

Return a list of orders. For each order return a record for each order detail within the order.

In [48]:
cur.execute('''select orderNumber from orders;''')
df = pd.DataFrame(cur.fetchall())
print(len(df))

326


In [52]:
cur.execute('''select * from orderdetails;''')
df = pd.DataFrame(cur.fetchall())
print(len(df))
df.head()

2996


,0,1,2,3,4
0,10100,S18_1749,30,136.00,3
1,10100,S18_2248,50,55.09,2
2,10100,S18_4409,22,75.46,4
3,10100,S24_3969,49,35.29,1
4,10101,S18_2325,25,108.06,4


In [57]:
cur.execute('''select orderNumber, productCode, quantityOrdered, priceEach, orderLineNumber
                from orders
                left join 
                orderdetails using(orderNumber)
                group by 1
                order by 1;''')
df = pd.DataFrame(cur.fetchall())
df.columns = [i[0] for i in cur.description]
print(len(df))
df.head()

326


,orderNumber,productCode,quantityOrdered,priceEach,orderLineNumber
0,10100,S24_3969,49,35.29,1
1,10101,S24_2022,46,44.35,2
2,10102,S18_1367,41,43.13,1
3,10103,S700_2824,42,94.07,6
4,10104,S50_1514,32,53.31,2


## Orders, Order details and Product Details (a Many-to-Many Join)

Return a list of the orders with the the order information, and allthe product information for each product in the order. 

In [58]:
cur.execute('''select * from products;''')
df = pd.DataFrame(cur.fetchall())
df.columns = [i[0] for i in cur.description]
print(len(df))
df.head()

110


,productCode,productName,productLine,productScale,productVendor,productDescription,quantityInStock,buyPrice,MSRP
0,S10_1678,1969 Harley Davidson Ultimate Chopper,Motorcycles,1:10,Min Lin Diecast,"This replica features working kickstand, front...",7933,48.81,95.70
1,S10_1949,1952 Alpine Renault 1300,Classic Cars,1:10,Classic Metal Creations,Turnable front wheels; steering function; deta...,7305,98.58,214.30
2,S10_2016,1996 Moto Guzzi 1100i,Motorcycles,1:10,Highway 66 Mini Classics,"Official Moto Guzzi logos and insignias, saddl...",6625,68.99,118.94
3,S10_4698,2003 Harley-Davidson Eagle Drag Bike,Motorcycles,1:10,Red Start Diecast,"Model features, official Harley Davidson logos...",5582,91.02,193.66
4,S10_4757,1972 Alfa Romeo GTA,Classic Cars,1:10,Motor City Art Classics,Features include: Turnable front wheels; steer...,3252,85.68,136.00


In [60]:
cur.execute('''select * 
                from orders
                left join 
                orderdetails using(orderNumber)
                left join 
                products using (productCode)
                group by orderNumber
                order by orderNumber;''')
df = pd.DataFrame(cur.fetchall())
df.columns = [i[0] for i in cur.description]
print(len(df))
df.head()

326


,orderNumber,orderDate,requiredDate,shippedDate,status,comments,customerNumber,productCode,quantityOrdered,priceEach,orderLineNumber,productName,productLine,productScale,productVendor,productDescription,quantityInStock,buyPrice,MSRP
0,10100,2003-01-06,2003-01-13,2003-01-10,Shipped,,363,S24_3969,49,35.29,1,1936 Mercedes Benz 500k Roadster,Vintage Cars,1:24,Red Start Diecast,This model features grille-mounted chrome horn...,2081,21.75,41.03
1,10101,2003-01-09,2003-01-18,2003-01-11,Shipped,Check on availability.,128,S24_2022,46,44.35,2,1938 Cadillac V-16 Presidential Limousine,Vintage Cars,1:24,Classic Metal Creations,This 1:24 scale precision die cast replica of ...,2847,20.61,44.80
2,10102,2003-01-10,2003-01-18,2003-01-14,Shipped,,181,S18_1367,41,43.13,1,1936 Mercedes-Benz 500K Special Roadster,Vintage Cars,1:18,Studio M Art Models,This 1:18 scale replica is constructed of heav...,8635,24.26,53.91
3,10103,2003-01-29,2003-02-07,2003-02-02,Shipped,,121,S700_2824,42,94.07,6,1982 Camaro Z28,Classic Cars,1:18,Carousel DieCast Legends,Features include opening and closing doors. Co...,6934,46.53,101.15
4,10104,2003-01-31,2003-02-09,2003-02-01,Shipped,,141,S50_1514,32,53.31,2,1962 City of Detroit Streetcar,Trains,1:50,Classic Metal Creations,This streetcar is a joy to see. It has 99 sepa...,1645,37.49,58.58


## Summary

In this lab, you practiced your knowledge on One-to-Many and Many-to-many relationships!